## Actionable Insights On New Movies

Please fill out:
* Anthony Brocco 
* Full Time NYC-DS-080723
* Phase 2  
* Joe Matta

## In this jupyter notebook we explore movie datasets from:

Box Office https://www.boxofficemojo.com/

IMDB https://www.imdb.com/

Rotten Tomatoes https://www.rottentomatoes.com/

TheMovieDB https://www.themoviedb.org/

The Numbers https://www.the-numbers.com/

In [2]:
import pandas as pd
import sqlite3


In [11]:
df_budget = pd.read_csv('../zippedData/movie_budget_cleaned.csv', index_col = 0)
df_budget

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,ROI
0,1,"Dec 18, 2009",Avatar,425000000,760507625,"$2,776,345,279",5.532577
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,"$1,045,663,875",1.546673
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,"$149,762,350",-0.572108
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,"$1,403,013,963",3.243841
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,"$1,316,721,747",3.153696
...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,$0,-1.000000
5778,79,"Apr 2, 1999",Following,6000,48482,"$240,495",39.082500
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,"$1,338",-0.732400
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,$0,-1.000000


In [14]:
df_successful = df_budget[df_budget['ROI'] > 1]
df_successful

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,ROI
0,1,"Dec 18, 2009",Avatar,425000000,760507625,"$2,776,345,279",5.532577
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,"$1,045,663,875",1.546673
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,"$1,403,013,963",3.243841
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,"$1,316,721,747",3.153696
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,306000000,936662225,"$2,053,311,220",5.710167
...,...,...,...,...,...,...,...
5773,74,"Feb 26, 1993",El Mariachi,7000,2040920,"$2,041,928",290.704000
5774,75,"Oct 8, 2004",Primer,7000,424760,"$841,926",119.275143
5775,76,"May 26, 2006",Cavite,7000,70071,"$71,644",9.234857
5778,79,"Apr 2, 1999",Following,6000,48482,"$240,495",39.082500


In [43]:
df_basics = pd.read_sql("""
    SELECT r.movie_id, primary_title, original_title, genres, averagerating, numvotes, start_year, runtime_minutes
FROM movie_ratings AS r
JOIN movie_basics AS b ON r.movie_id = b.movie_id;

""", conn)

df_basics

,movie_id,primary_title,original_title,genres,averagerating,numvotes,start_year,runtime_minutes
0,tt10356526,Laiye Je Yaarian,Laiye Je Yaarian,Romance,8.3,31,2019,117.0
1,tt10384606,Borderless,Borderless,Documentary,8.9,559,2019,87.0
2,tt1042974,Just Inès,Just Inès,Drama,6.4,20,2010,90.0
3,tt1043726,The Legend of Hercules,The Legend of Hercules,"Action,Adventure,Fantasy",4.2,50352,2014,99.0
4,tt1060240,Até Onde?,Até Onde?,"Mystery,Thriller",6.5,21,2011,73.0
...,...,...,...,...,...,...,...,...
73851,tt9805820,Caisa,Caisa,Documentary,8.1,25,2018,84.0
73852,tt9844256,Code Geass: Lelouch of the Rebellion - Glorifi...,Code Geass: Lelouch of the Rebellion Episode III,"Action,Animation,Sci-Fi",7.5,24,2018,120.0
73853,tt9851050,Sisters,Sisters,"Action,Drama",4.7,14,2019,NaN
73854,tt9886934,The Projectionist,The Projectionist,Documentary,7.0,5,2019,81.0


In [ ]:
df_bas

In [ ]:
# Assuming 'column_to_drop' is the name of the column you want to drop



In [19]:
pd.read_sql("""
    SELECT *
    FROM movie_basics
""", conn).columns

Index(['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres'],
      dtype='object')

## Business Questions
What are the top-grossing movie genres in the current market, and how have their popularity trends evolved over the past few years?

Which demographic groups are driving the highest box office revenue, and what genres or themes resonate most with these audiences?

Can we identify any successful trends or patterns in the distribution and marketing strategies of recent blockbuster movies?

What are the most lucrative international markets for Hollywood films, and how can we tailor our content to appeal to these diverse audiences?

Are there any untapped niche genres or storytelling approaches that have shown potential for profitability in the modern movie landscape?

How do critical acclaim and audience reception correlate with box office success, and can we use this information to refine our content development and marketing strategies?

What are the emerging technologies and platforms (e.g., streaming services, virtual reality) that are shaping the future of the film industry, and how can we leverage them to our advantage?

What is the average budget range for successful movies in various genres, and how can we allocate our resources effectively to maximize ROI?

Who are the rising talents, directors, writers, and actors in the industry, and how can we collaborate with them to enhance the appeal of our productions?

What is the competitive landscape like in the movie industry, and what unique selling propositions can our studio develop to stand out and capture market share?